In [ ]:
# !pip install google-generativeai faiss-cpu langchain
# !pip install langchain-google-genai
# !pip install --upgrade google-generativeai langchain langchain-google-genai

  Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl.metadata (5.7 kB)
Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl (1.3 MB)
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.7.0
    Uninstalling google-ai-generativelanguage-0.7.0:
      Successfully uninstalled google-ai-generativelanguage-0.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 2.1.12 requires google-ai-generativelanguage<1,>=0.7, but you have google-ai-generativelanguage 0.6.15 which is incompatible.


In [ ]:
import os
API_KEY = os.getenv("GOOGLE_GEMINI_API_KEY")
os.environ["GOOGLE_API_KEY"] = API_KEY 
# GOOGLE_GEMINI_API_KEY = ""
# os.environ["GOOGLE_API_KEY"] = GOOGLE_GEMINI_API_KEY

In [51]:

import google.generativeai as genai
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader, PyPDFLoader
# from langchain_community.embeddings import GoogleGenerativeAIEmbedding
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [ ]:
# 🔑 Configure Gemini
genai.configure(api_key=GOOGLE_GEMINI_API_KEY)

# check model
model = genai.GenerativeModel('gemini-1.5-flash-latest')
# Send a prompt to the model
response = model.generate_content("Explain quantum computing in a simple paragraph.")
# Print the response
print(response.text)

E0000 00:00:1758385677.294587   11372 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


Quantum computing harnesses the bizarre principles of quantum mechanics to solve problems too complex for even the most powerful classical computers.  Instead of bits representing 0 or 1, it uses qubits, which can be 0, 1, or a combination of both simultaneously (superposition).  This, along with other quantum phenomena like entanglement (linking qubits' fates), allows quantum computers to explore many possibilities at once, potentially revolutionizing fields like medicine, materials science, and artificial intelligence.



In [ ]:
# Step 1: Load knowledge base
documents = []
for root, _, files in os.walk("./knowledge"):
    for file in files:
        path = os.path.join(root, file)
        if file.lower().endswith(".pdf"):
            pdf_loader = PyPDFLoader(path)
            documents.extend(pdf_loader.load())
        elif file.lower().endswith(".txt"):
            text_loader = TextLoader(path)
            documents.extend(text_loader.load())

print(f"Loaded {len(documents)} documents.")

# Step 2: Split into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(documents)
print(f"Split into {len(chunks)} chunks.")

Loaded 3 documents.
Split into 19 chunks.


In [63]:
# Step 3: Use Gemini embeddings + FAISS

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001", 
    google_api_key=GOOGLE_GEMINI_API_KEY
)
vector_store = FAISS.from_documents(chunks, embeddings)
retriever = vector_store.as_retriever(search_kwargs={"k": 2})

E0000 00:00:1758386819.049817   11372 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


GoogleGenerativeAIError: Error embedding content: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.
* Quota exceeded for metric: generativelanguage.googleapis.com/embed_content_free_tier_requests, limit: 0
* Quota exceeded for metric: generativelanguage.googleapis.com/embed_content_free_tier_requests, limit: 0
* Quota exceeded for metric: generativelanguage.googleapis.com/embed_content_free_tier_requests, limit: 0
* Quota exceeded for metric: generativelanguage.googleapis.com/embed_content_free_tier_requests, limit: 0 [violations {
  quota_metric: "generativelanguage.googleapis.com/embed_content_free_tier_requests"
  quota_id: "EmbedContentRequestsPerMinutePerProjectPerModel-FreeTier"
}
violations {
  quota_metric: "generativelanguage.googleapis.com/embed_content_free_tier_requests"
  quota_id: "EmbedContentRequestsPerMinutePerUserPerProjectPerModel-FreeTier"
}

In [ ]:
# Use HuggingFace embeddings + Gemini for LLM
# gemini models/embedding-001 not working properly

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI

# Use HuggingFace embeddings locally
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_store = FAISS.from_documents(chunks, embeddings)

retriever = vector_store.as_retriever(search_kwargs={"k": 2})

In [58]:
# Use Gemini for generation
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=GOOGLE_GEMINI_API_KEY)


E0000 00:00:1758386223.737971   11372 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [66]:
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

prompt_template = """
You are a helpful assistant. 
Answer the user question using only the following context: 
{context}

Question: {question}
If the answer is not in the context, say "I don't know".
"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [67]:
question = "Who is Dasun?"
answer = rag_chain.invoke(question)
print("Answer:", answer)

Answer: Dasun Pathirage is an experienced Senior Software Engineer with a robust background in embedded software development, C++, FPGA design, RISC-V architecture, and machine learning.  He is recognized for his quick learning ability, strong analytical thinking, and superior problem-solving skills.
